In [1]:
import pandas as pd
import xlrd
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import matplotlib.pyplot as plt
import plotly.express as px
from datetime import datetime
import os
from pathlib import Path

In [2]:
current_dir = Path.cwd()

# Include both .xls and .xlsx
excel_files = sorted([
    f.name for f in current_dir.iterdir() 
    if f.is_file() and f.suffix.lower() in ['.xls', '.xlsx']
])

print(f"📁 Found {len(excel_files)} Excel files (.xls + .xlsx)")

# Widgets
file_dropdown = widgets.Dropdown(
    options=excel_files,
    description='Select File:',
    value='AA_OpenMeFirst.xls' if 'AA_OpenMeFirst.xls' in excel_files else excel_files[0] if excel_files else None,
    layout=widgets.Layout(width='80%')
)
sheet_dropdown = widgets.Dropdown(description='Select Sheet:', options=[])

assess_btn = widgets.Button(description="Show Structure", button_style='primary', icon='table')
extract_single_btn = widgets.Button(description="Extract This File", button_style='success', icon='file')
batch_btn = widgets.Button(description="🚀 Batch Generate All CSVs", button_style='danger', icon='cogs')

output = widgets.Output()

def update_sheets(change):
    if not file_dropdown.value:
        return
    file_path = current_dir / file_dropdown.value
    try:
        if file_path.suffix.lower() == '.xls':
            wb = xlrd.open_workbook(str(file_path))
            sheet_dropdown.options = wb.sheet_names()
        else:
            xls = pd.ExcelFile(str(file_path))
            sheet_dropdown.options = xls.sheet_names
    except Exception as e:
        sheet_dropdown.options = []
        print("Error:", e)

file_dropdown.observe(update_sheets, names='value')
if file_dropdown.value:
    update_sheets(None)

def show_structure(b):
    with output:
        clear_output()
        file_path = current_dir / file_dropdown.value
        print(f"📂 {file_dropdown.value}")
        try:
            if file_path.suffix.lower() == '.xls':
                wb = xlrd.open_workbook(str(file_path))
                print(f"✅ .xls → {len(wb.sheet_names())} sheets")
                sheet_name = sheet_dropdown.value or wb.sheet_names()[0]
                df = pd.read_excel(str(file_path), sheet_name=sheet_name, engine='xlrd', nrows=15)
            else:
                df = pd.read_excel(str(file_path), nrows=15)
                print("✅ .xlsx")
            
            display(df.head(10))
        except Exception as e:
            print("❌ Error:", e)

def extract_single(b):
    with output:
        clear_output()
        file_path = current_dir / file_dropdown.value
        print(f"🔧 Extracting BCA-relevant data from {file_dropdown.value}")
        try:
            wb = xlrd.open_workbook(str(file_path))
            all_data = {}
            
            for sheet_name in wb.sheet_names():
                df = pd.read_excel(str(file_path), sheet_name=sheet_name, engine='xlrd')
                print(f"\n→ Sheet '{sheet_name}': {df.shape}")
                
                # Try to clean common headers (skip empty rows/columns)
                df = df.dropna(how='all').reset_index(drop=True)
                if len(df) > 0 and df.iloc[0].isnull().sum() > len(df.columns)/2:
                    df.columns = df.iloc[0]
                    df = df[1:].reset_index(drop=True)
                
                all_data[sheet_name] = df
                
                # Save each sheet as clean CSV
                clean_name = f"{file_dropdown.value.replace('.xls','')}_{sheet_name.replace(' ','_')}.csv"
                df.to_csv(clean_name, index=False)
                print(f"   Saved → {clean_name} ({len(df)} rows)")
            
            print("\n✅ Extraction complete! Check the files in your folder.")
            
        except Exception as e:
            print("❌ Error:", e)

def batch_generate_all(b):
    with output:
        clear_output()
        print(f"🚀 Batch processing {len(excel_files)} files (.xls + .xlsx)...\n")
        start_time = datetime.now()
        summary = []
        success = 0

        # Create main extraction folder
        extract_folder = current_dir / "BCA_Extracted"
        extract_folder.mkdir(exist_ok=True)
        print(f"📂 Created extraction folder: {extract_folder.name}/\n")

        for fname in excel_files:
            file_path = current_dir / fname
            ext = file_path.suffix.lower()
            
            # Create subfolder for this Excel file
            file_base_name = fname.replace(ext, '')
            file_folder = extract_folder / file_base_name
            file_folder.mkdir(exist_ok=True)
            
            print(f"Processing: {fname}")

            try:
                if ext == '.xls':
                    wb = xlrd.open_workbook(str(file_path))
                    sheets = wb.sheet_names()
                    engine = 'xlrd'
                else:
                    xls = pd.ExcelFile(str(file_path))
                    sheets = xls.sheet_names
                    engine = None  # default = openpyxl

                for sheet_name in sheets:
                    try:
                        df = pd.read_excel(str(file_path), sheet_name=sheet_name, engine=engine)
                        df = df.dropna(how='all').reset_index(drop=True)

                        # Auto-promote header if first row looks like header
                        if len(df) > 0 and df.iloc[0].isnull().sum() > len(df.columns) * 0.5:
                            df.columns = df.iloc[0]
                            df = df[1:].reset_index(drop=True)

                        # Save to the file's dedicated subfolder
                        clean_name = f"{file_base_name}_{sheet_name.replace(' ', '_')}.csv"
                        output_path = file_folder / clean_name
                        df.to_csv(output_path, index=False)

                        summary.append({
                            'File': fname,
                            'Sheet': sheet_name,
                            'Rows': len(df),
                            'Cols': len(df.columns),
                            'Output': f"{file_base_name}/{clean_name}"
                        })
                        print(f"   ✅ {sheet_name} → {len(df)} rows")
                    except Exception as se:
                        print(f"   ⚠️  {sheet_name}: {se}")

                success += 1
            except Exception as e:
                print(f"   ❌ {fname}: {e}")

        # Save summary in the extraction folder
        summary_df = pd.DataFrame(summary)
        summary_df.to_excel(extract_folder / "BCA_Batch_Summary_Report.xlsx", index=False)
        summary_df.to_csv(extract_folder / "BCA_Batch_Summary_Report.csv", index=False)

        elapsed = datetime.now() - start_time
        print(f"\n✅ Batch complete in {elapsed.seconds}s")
        print(f"   Successful files: {success}/{len(excel_files)}")
        print(f"   Total sheets extracted: {len(summary_df)}")
        print(f"   All files saved to: {extract_folder.name}/")
        display(summary_df.head(20))

# Button bindings
assess_btn.on_click(show_structure)
extract_single_btn.on_click(extract_single)
batch_btn.on_click(batch_generate_all)

display(widgets.VBox([
    widgets.HTML("<h3>BCA Extractor - Supports .xls + .xlsx</h3>"),
    file_dropdown, sheet_dropdown,
    widgets.HBox([assess_btn, extract_single_btn, batch_btn]),
    output
]))

📁 Found 15 Excel files (.xls + .xlsx)


In [5]:
import pandas as pd
from pathlib import Path

control_path = Path.cwd()/ "BCA_Extracted" / "AA_OpenMeFirst" / "AA_OpenMeFirst_Control.csv"
df = pd.read_csv(control_path)

✅ Extracted 0 daily fuel records


/tmp/ipykernel_1114/3653759764.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  'Date': pd.to_datetime(data_rows['Unnamed: 42'], errors='coerce'),


,Date,Pertamina_Fuel_Price_Rp,Exchange_Rate_USD_IDR



✅ Extracted distances (updated with PDF info):


,Location,Distance_NM
0,DOBS_South_Area,14
1,DOBU_North_Area,125
2,Sapi,70
3,Santan_Terminal,125
4,Sesulu,40
5,Makassar,126
6,Ganal,50-165
7,Rapak,50-165



✅ Rigs Summary:


,Active_Rigs,Inactive_Rigs
0,NaN,0



Period: 25 Nov - 24 Dec 2025
